In [1]:
pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.9/722.9 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import numpy as np
import mlflow
import mlflow.spark

spark = SparkSession.builder.appName("Housing Price Prediction Pipeline").getOrCreate()

locations = ['Los Angeles, CA', 'San Francisco, CA', 'Austin, TX', 'Miami, FL',
             'New York, NY', 'Denver, CO', 'Chicago, IL', 'Seattle, WA',
             'Atlanta, GA', 'Boston, MA']
nearby_transport_options = ['Yes', 'No']

data = []
for _ in range(10000):
    location = np.random.choice(locations)
    bedrooms = int(np.random.randint(1, 5))
    bathrooms = int(np.random.randint(1, 4))
    size_sqft = float(np.random.normal(loc=1500, scale=300))
    lot_size_sqft = float(np.random.normal(loc=5000, scale=1000))
    days_on_market = int(np.random.randint(10, 90))
    interest_rate = float(np.random.uniform(2.5, 5.0))
    median_income = float(np.random.normal(loc=80000, scale=15000))
    school_rating = int(np.random.randint(5, 10))
    walkability_score = int(np.random.randint(50, 100))
    nearby_transport = np.random.choice(nearby_transport_options)
    sale_price = (100000 +
                  bedrooms * 30000 +
                  bathrooms * 20000 +
                  size_sqft * 200 +
                  lot_size_sqft * 10 +
                  median_income * 0.5 +
                  float(np.random.normal(0, 20000)))
    data.append((str(location), bedrooms, bathrooms, size_sqft, lot_size_sqft,
                 days_on_market, interest_rate, median_income, school_rating,
                 walkability_score, str(nearby_transport), float(sale_price)))

columns = ['Location', 'Bedrooms', 'Bathrooms', 'Size_SqFt', 'Lot_Size_SqFt',
           'Days_on_Market', 'Interest_Rate', 'Median_Income', 'School_Rating',
           'Walkability_Score', 'Nearby_Transport', 'Sale_Price']

df = spark.createDataFrame(data, columns)

indexer_location = StringIndexer(inputCol="Location", outputCol="Location_Index")
indexer_transport = StringIndexer(inputCol="Nearby_Transport", outputCol="Transport_Index")

feature_columns = ['Location_Index', 'Bedrooms', 'Bathrooms', 'Size_SqFt',
                   'Lot_Size_SqFt', 'Days_on_Market', 'Interest_Rate',
                   'Median_Income', 'School_Rating', 'Walkability_Score', 'Transport_Index']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
lr = LinearRegression(featuresCol="scaled_features", labelCol="Sale_Price")

pipeline = Pipeline(stages=[indexer_location, indexer_transport, assembler, scaler, lr])
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
pipeline_model = pipeline.fit(train_df)

predictions = pipeline_model.transform(test_df)
evaluator = RegressionEvaluator(labelCol="Sale_Price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Use MLflow to log and register the model
with mlflow.start_run() as run:
    mlflow.log_metric("rmse", rmse)
    mlflow.spark.log_model(pipeline_model, "housing_price_pipeline")
    run_id = run.info.run_id

    model_uri = f"runs:/{run_id}/housing_price_pipeline"
    print(model_uri)
    registered_model = mlflow.register_model(model_uri, "HousingPricePredictionPipeline")

spark.stop()


Root Mean Squared Error (RMSE): 19957.95006190776


2025/05/30 00:22:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'HousingPricePredictionPipeline'.
Created version '1' of model 'HousingPricePredictionPipeline'.


runs:/449387891dc94114a7a1f3cf1c67cea8/housing_price_pipeline


In [8]:
!nohup mlflow models serve --model-uri runs:/{run_id}/housing_price_pipeline --port 5000 --no-conda 0<&- &>/dev/null &

In [9]:
import requests
import json
import time

data = {
    "dataframe_split": {
        "columns": [
            "Location_Index", "Bedrooms", "Bathrooms", "Size_SqFt", "Lot_Size_SqFt",
            "Days_on_Market", "Interest_Rate", "Median_Income", "School_Rating",
            "Walkability_Score", "Transport_Index"
        ],
        "data": [
            [3.0, 3, 2, 1600, 5500, 30, 3.5, 75000, 8, 80, 1.0]
        ]
    }
}

response = requests.post("http://127.0.0.1:5000/invocations", json=data)
print("Predictions:", response.json())


Predictions: {'predictions': [642280.5505069736]}
